# Introduction to `pandas` and `matplotlib` with example data from USGS's `dataretrieval`

`pandas` facilitates a lot of data analysis including the powerful [`groupby()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html), which helps you use the ["split-apply-combine"](https://pandas.pydata.org/docs/user_guide/groupby.html) method of data analysis

`matplotlib` is a popular and easy to use plotting library for Ptyhon. It resembles MATLAB plotting tools. 

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np # numpy is where all the math happens

The US Geologic Survey is making it easier to find and retrieve their data. Most development so far has been in R, but a few blessed individuals have been developing a similar package in Python, [dataretrieval](https://github.com/USGS-python/dataretrieval). Read more about [automated data retrieval](https://waterservices.usgs.gov/rest/Site-Service.html). Since I mocked up this activity for EARS33 in March 2022, there are a lot of new features and [demos](https://github.com/DOI-USGS/dataretrieval-python/tree/master/demos/hydroshare) on the repo. 


In [ ]:
from dataretrieval import nwis

# Part 1: EARS33 example activity

For this activity, I demonstrate the use of nested watersheds and their gages. I grabbed three gages from nested watersheds by loookin at [this map](https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default). 

FYI I haven't found an easy web interface where you can see the watersheds of individual gages but the shapefiles I think are [here](https://water.usgs.gov/GIS/metadata/usgswrd/XML/gagesII_Sept2011.xml).

In [ ]:
# Make a list of strings corresponding to site IDs. 
# These use quotes to denote they are strings and not numbers
# They are separated by commas and bound by square brackets

sitelist = ["01075000", "01081500", "01092000"]


In [ ]:
site_info = nwis.get_record(sites=sitelist, service='site')

print('Available data are:\n', site_info.columns.values) # "\n" just puts a line break in a text (string)
print('Station names are:\n', site_info['station_nm'])


Now to Pandas

In [ ]:
# Initialize a blank dataframe object
df = pd.DataFrame()

Parameter codes for the USGS are here:https://help.waterdata.usgs.gov/codes-and-parameters/parameters To get discharge, use "00060"

In [ ]:
# This loop will iterate through each of the objects in sitelist and assign it 
# a number "i" as it assigns the variable siteNumber to the sitelist object

for i, siteNumber in enumerate(sitelist):
    # This is the parameter code for discharge in cfs
    parameterCode = "00060"
    # These are strings of dates
    startDate = "2022-01-01"
    endDate = "2022-12-31"
    # Make a temporary dataframe to store the records of each site
    df_temp = nwis.get_record(sites=siteNumber, service='dv', start=startDate, end=endDate, parameterCd='00060')
    # And then append each site's data to the previous sites' data
    df = df.append(df_temp)
    # This method keeps our dataframe compact - each of the sites have the same
    # data, and we can always parse by site number later

In [ ]:
df

Provisional data might be bad numbers for discharge:

In [ ]:
df.groupby(by='site_no')['00060_Mean'].min()

-999999.0 isn't real, so let's replace with NaN (not zero!)

In [ ]:
df.replace(-999999.0, np.nan, inplace=True)

In [ ]:
df.groupby(by='site_no')['00060_Mean'].min()

Now we're going to do a groupby that will [group rows with similar values](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) for a certain column for analysis. After we groupby, we can query the values of a different column, but grouped by the first column.

In this case the code is going to make three separate plots, one for each site_no, of the column 00060_Mean, or mean daily discharge.





In [ ]:
fig, ax = plt.subplots(figsize=(15,4))

for site, group in df.groupby(by='site_no'):
    
    # site_info = nwis.get_record(sites=site, service='site')
    # To do - get name and drainage area as label instead of station label!

    group.plot(y='00060_Mean', ax=ax, legend=True, label=site)

ax.set_xlabel('Date')
ax.set_ylabel('Streamflow (cfs)')

#https://stackoverflow.com/questions/39902522/pandas-groupby-object-in-legend-on-plot

Very cool!

1. Which plot is the lowest drainage area? The highest? How do you know?
2. How might you write an algorithm to determine the timing between peak discharges between upstream and downstream sites?
3. Are the upstream sites always a consistent percentage of the downstream sites' discharges? Why or why not? How might that be explained? What other data might you need to test this idea? BONUS: How might you write an algorithm to track how the relative streamflows change with time? 

# Part 2: discharge trends in Alaskan rivers

So I used this search page to create a table of all Alaska gage sites and then use Pandas to parse the resulting table: https://waterdata.usgs.gov/ak/nwis/current?submitted_form=introduction

In [ ]:
import requests

In [ ]:
url = 'https://waterdata.usgs.gov/ak/nwis/current?index_pmcode_STATION_NM=1&index_pmcode_DATETIME=2&group_key=basin_cd&format=sitefile_output&sitefile_output_format=html_table&column_name=agency_cd&column_name=site_no&column_name=station_nm&sort_key_2=site_no&html_table_group_key=NONE&rdb_compression=file&list_of_search_criteria=realtime_parameter_selection'

usgs_url = requests.get(url)

[`pd.read_html()`](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html) parses for tables, which is why I generated a table (and not the tab-delimeted text, which seems like that should have worked but whatever)

In [ ]:
# This returns a LIST of dataframes
usgs_data = pd.read_html(usgs_url.text)

In [ ]:
# So let's look at the first (and only) dataframe that comes out
usgs_data = usgs_data[0]

usgs_data.head()

In [ ]:
# get_record takes a string
site_string = str(usgs_data.iloc[0]["Site Number"])
# And iloc[0] is "the data at index location 0"
site_info = nwis.get_record(sites=site_string, service='site')
print("Lat: ", site_info['dec_lat_va'])
print("Long: ", site_info['dec_long_va'])


In [ ]:
# Get the discharge history for the site and clean bad data
df_hist = nwis.get_record(sites=site_string, service='dv', start='1900-01-01', end=endDate, parameterCd='00060')
df_hist.replace(-999999.0, np.nan, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(15,4))
data = df_hist.plot(y='00060_Mean', ax=ax)
ax.set_xlabel('Date')
ax.set_ylabel('Streamflow (cfs)')

In [ ]:
#pandas datetimeindex docs: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.html
df_hist['year'] = pd.DatetimeIndex(pd.to_datetime(df_hist.index)).year

# This is a slightly weird thing, where the datetime is stored as an "index"
# of the dataframe rather than a column, so I first turned the index into 
# a datetime object with the pd.to_datetime(df_hist.index) call, 
# and then parsed that datetime object for the year

df_hist.head() 

In [ ]:
# Here I'm going to ask "What is the maximum value for each year?" the groupby()
# function puts all the things that have the same value in a specified column
# and then finds the max value, in this case 
# Read more here: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

annual_max_floods = df_hist.groupby('year').max().reset_index()

# The groupby() function creates this new mini-dataframe, "annual_max_floods",
# which can be manipulated just like our original dataframe, df_hist


# the reset_index() call is just a little nuance where I don't want it to turn
# my year groups into the index of the dataframe, or else Pandas will thing
# I want to plot variables as a timeline, which I don't want to do 

annual_max_floods

In [ ]:
# Here I'm asking Pandas to rank the discharge column in descending order...

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rank.html
annual_max_floods['rank'] = annual_max_floods['00060_Mean'].rank(ascending=False)

# ...and then show them to me sorted!

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
annual_max_floods.sort_values(by=['rank'])

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.scatter.html
annual_max_floods.plot.scatter(x='rank',y='00060_Mean')

# Why don't you edit the above code to:
# Properly label x and y axes
# maybe...add the year as a color for the data points?

Maybe this is the time to write a function that does all that work for you "under the hood" rather than in big code blocks because you'll want to do the same thing to the data over and over again

In [ ]:
def get_annual_floods(site_string):
  # site_string is the string of the gage
  site_info = nwis.get_record(sites=site_string, service='site')
  
  df_hist = nwis.get_record(sites=site_string, service='dv', start='1900-01-01', end=endDate, parameterCd='00060')
  df_hist.replace(-999999.0, np.nan, inplace=True)

  df_hist['year'] = pd.DatetimeIndex(pd.to_datetime(df_hist.index)).year

  annual_max_floods = df_hist.groupby('year').max().reset_index()

  annual_max_floods['rank'] = annual_max_floods['00060_Mean'].rank(ascending=False)

  # And then this is the output to your function: a sorted dataframe
  return annual_max_floods.sort_values(by=['rank'])


Can you add some lines to the function that perhaps print the name of the site when you call `get_annual_floods()`?

In [ ]:
max_flood_data = get_annual_floods(str(usgs_data.iloc[0]["Site Number"]))
max_flood_data.plot.scatter(x='rank',y='00060_Mean', c='year', cmap='viridis')

In [ ]:
max_flood_data = get_annual_floods(str(usgs_data.iloc[1]["Site Number"]))
max_flood_data.plot.scatter(x='rank',y='00060_Mean', c='year', cmap='viridis')

In [ ]:
max_flood_data = get_annual_floods(str(usgs_data.iloc[2]["Site Number"]))
max_flood_data.plot.scatter(x='rank',y='00060_Mean', c='year', cmap='viridis')

Fun!!

# Part 3: Hackathon prompt

One of the things I wonder in my work is based on climate change, will the biggest discharge events (which have historically occurred early in the growing season and are associated with snowmelt) instead be associated with rainstorms later in the growing season, when the ground is more thawed?

Hackathon prompt: can you test the hypothesis that ***maximum floods are occurring later in the year?*** And **are those floods getting bigger**?

Ingredients:


1.   Sitewise parsing of maximum flood dates
2.   Some means of quantifying the trend in date of the peak flood (perhaps a [regression line fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html) to the day of year of max flood?
3.   Some means of reducing that quantification of trend down to a single number or variable that can be added to the `usgs_data` dataframe 

